In [ ]:
import ezdxf
import vtk

# Get DXF data 

Here we assume that wireframes are stored as `3DFACE` entities

In [ ]:
dwg = ezdxf.readfile('test.dxf')

In [ ]:
# iterate over all entities in model space
msp = dwg.modelspace()

# entity query for all LINE entities in model space
triangles = []
for e in msp.query('3DFACE'):
    # get point coordinates of the triangle (one per row)
    triangles.append([e.dxf.vtx0, e.dxf.vtx1, e.dxf.vtx3])


In [ ]:
#first triangle
print (triangles[0])

# Put Triangles in VTK

In [ ]:
# Setup points
meshpoints = vtk.vtkPoints()
meshtriangle = vtk.vtkTriangle()
meshtriangles = vtk.vtkCellArray()
id = 0
for i in triangles:
    meshpoints.InsertNextPoint(i[0])
    meshpoints.InsertNextPoint(i[1])
    meshpoints.InsertNextPoint(i[2])
    meshtriangle = vtk.vtkTriangle()
    meshtriangle.GetPointIds().InsertId(0,id)
    meshtriangle.GetPointIds().InsertId(1,id+1)
    meshtriangle.GetPointIds().InsertId(2,id+2)
    meshtriangles.InsertNextCell(meshtriangle)
    id = id+3

mesh = vtk.vtkPolyData()    
mesh.SetPoints(meshpoints)
mesh.SetPolys(meshtriangles)

# doing a bit of cleanup (remove duplicated points)
cleanPolyData = vtk.vtkCleanPolyData()
cleanPolyData.SetInputData(mesh)
cleanPolyData.Update()


In [ ]:
#export mesh
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName("test.vtp")
writer.SetInputData(cleanPolyData.GetOutput())
writer.Write()
